In [4]:
from pymongo import MongoClient
import polars as pl

In [5]:
# Conectar ao MongoDB
client = MongoClient("mongodb://localhost:27017")  # Conectar ao MongoDB local
db = client["ctt2025"]  # Nome da base de dados no MongoDB
collection = db["expresso_2025"]  # Nome da coleção (tabela)

In [6]:
from datetime import datetime, timedelta, timezone
import polars as pl

# --- intervalo últimos 30 dias ---
end = datetime.now(timezone.utc)
start = end - timedelta(days=30)

DATE_FIELD = "data_criacao"

proj = {
    "centro": 1, "data_criacao": 1, "giro": 1, "loptica": 1,
    "janela_horaria": 1, "nome": 1, "morada": 1, "cp": 1,
    "localidade": 1, "cod_t_even": 1, "data_evento": 1,
    "latitude": 1, "longitude": 1, "nome_rem": 1, "cod_pais_origem": 1,
    "_id": 0  # ignora o _id, se não precisares
}

query = {DATE_FIELD: {"$gte": start, "$lt": end}}

n = collection.count_documents(query)
print(f"Documentos no último mês: {n}")

cursor = collection.find(query, projection=proj, batch_size=10_000)

batches, batch = [], []
BATCH_SIZE = 50_000

for i, item in enumerate(cursor, 1):
    batch.append({
        "data_criacao": str(item.get("data_criacao") or item.get("DATA_CRIACAO") or ""),
        "centro": str(item.get("centro") or item.get("CENTRO") or ""),
        "giro": str(item.get("giro") or item.get("GIRO") or ""),
        "loptica": str(item.get("loptica") or item.get("LOPTICA") or ""),
        "janela_horaria": str(item.get("janela_horaria") or item.get("JANELA_HORARIA") or ""),
        "nome": str(item.get("nome") or item.get("NOME") or ""),
        "morada": str(item.get("morada") or item.get("MORADA") or ""),
        "cp": str(item.get("cp") or item.get("CP") or ""),
        "localidade": str(item.get("localidade") or item.get("LOCALIDADE") or ""),
        "cod_t_even": str(item.get("cod_t_even") or item.get("COD_T_EVEN") or ""),
        "data_evento": str(item.get("data_evento") or item.get("DATA_EVENTO") or ""),
        "latitude": str(item.get("latitude") or item.get("LATITUDE") or ""),
        "longitude": str(item.get("longitude") or item.get("LONGITUDE") or ""),
        "nome_rem": str(item.get("nome_rem") or item.get("NOME_REM") or ""),
        "cod_pais_origem": str(item.get("cod_pais_origem") or item.get("COD_PAIS_ORIGEM") or ""),
    })

    if i % BATCH_SIZE == 0:
        dfb = pl.DataFrame(batch)   # agora é seguro, tudo string
        batches.append(dfb)
        batch.clear()

if batch:
    dfb = pl.DataFrame(batch)
    batches.append(dfb)

df = pl.concat(batches, how="vertical_relaxed") if batches else pl.DataFrame()

# ✅ converter colunas de interesse depois
if df.height:
    df = df.with_columns([
        pl.col("data_criacao").str.strptime(pl.Datetime, strict=False),
        pl.col("latitude").cast(pl.Float64, strict=False),
        pl.col("longitude").cast(pl.Float64, strict=False),
    ])





Documentos no último mês: 8823102


In [7]:
df.head(50)

data_criacao,centro,giro,loptica,janela_horaria,nome,morada,cp,localidade,cod_t_even,data_evento,latitude,longitude,nome_rem,cod_pais_origem
datetime[μs],str,str,str,str,str,str,str,str,str,str,f64,f64,str,str
2025-08-02 23:00:00,"""8811373""","""2735H LUIS GONÇ""","""DS832898428PT""","""09-19""","""CARLA FLORENTINO""","""RUA DOS FARIAS 4""","""2735-504""","""AGUALVA-CACÉM""","""I""","""03/08/2025 19:29:49""",null,null,"""YUCAN - FULFILLMENT SERVICE FO…","""ES"""
2025-08-02 23:00:00,"""8811373""","""2665G NETCOUR""","""DD208879767PT""","""DOMINGO""","""BEATRIZ GRAA A""","""RUA DR JOSA SABINO LOURO, N. 1…","""2665-240""","""MALVEIRA""","""I""","""03/08/2025 19:23:07""",null,null,"""WORTEN - EQUIPAMENTO PARA O LA…","""PT"""
2025-08-02 23:00:00,"""8811373""","""2735K LUIS GONÇ""","""DW756915264PT""","""SABADO""","""JOAO ROMAO""","""PRACETA MARIA JUDITE DE CARVAL…","""2635-547""","""RIO DE MOURO""","""I""","""03/08/2025 19:20:26""",null,null,"""AMAZON ROAD TRANSPORT SPAIN, S…","""ES"""
2025-08-02 23:00:00,"""8811373""","""2735H LUIS GONÇ""","""DS832166365PT""","""09-19""","""ANDREIA SILVA""","""RUA CIDADE RIO DE JANEIRO, 25 …","""2735-659""","""AGUALVA-CACÉM""","""H""","""03/08/2025 19:18:42""",null,null,"""YUCAN - FULFILLMENT SERVICE FO…","""ES"""
2025-08-02 23:00:00,"""8811373""","""2735H LUIS GONÇ""","""DB189659077PT""","""09-19""","""RITA CORREIA""","""RUA DO LUSO N7, ENCOSTA S.MARC…","""2735-511""","""AGUALVA-CACÉM""","""H""","""03/08/2025 19:18:32""",null,null,"""TALICA RETROSARIA ADDRESS""","""PT"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-08-02 23:00:00,"""8811373""","""2735H LUIS GONÇ""","""DS831876823PT""","""09-19""","""ELIZABETE QUINTINHA""","""RUA SOCIEDADE RECREATIVA 13""","""2735-530""","""AGUALVA-CACÉM""","""I""","""03/08/2025 18:50:25""",null,null,"""YUCAN - FULFILLMENT SERVICE FO…","""ES"""
2025-08-02 23:00:00,"""8811373""","""2735K LUIS GONÇ""","""DW756964486PT""","""09-19""","""VIKTORIYA KURMAYEVA""","""ESTRADA MARQUES POMBAL - FITAR…","""2635-652""","""RIO DE MOURO""","""H""","""03/08/2025 18:49:43""",38.790249,-9.325962,"""AMAZON ROAD TRANSPORT SPAIN, S…","""ES"""
2025-08-02 23:00:00,"""8811373""","""2665G NETCOUR""","""DW757702708PT""","""09-19""","""ANDREIA ALEXANDRE""","""RUA DA FEITEIRA N 8 - ROLIA""","""2665-375""","""MILHARADO""","""I""","""03/08/2025 18:49:38""",null,null,"""AMAZON SPAIN FULFILLMENT, S.L.…","""ES"""


In [8]:
import polars as pl

# filtrar exatamente o dia 2025-08-25 (ignora horas)
filtro = df.filter(
    pl.col("data_criacao").dt.date() == pl.date(2025, 8, 20)
)


filtro


data_criacao,centro,giro,loptica,janela_horaria,nome,morada,cp,localidade,cod_t_even,data_evento,latitude,longitude,nome_rem,cod_pais_origem
datetime[μs],str,str,str,str,str,str,str,str,str,str,f64,f64,str,str
2025-08-20 23:00:00,"""8811361""","""MV260""","""RL313217931PT""","""""","""""","""""","""""","""""","""I""","""21/08/2025 17:41:21""",38.559422,-7.89567,"""""","""PT"""
2025-08-20 23:00:00,"""8811361""","""MV260""","""RL418074614PT""","""""","""""","""""","""""","""""","""I""","""21/08/2025 17:38:56""",38.560492,-7.89229,"""""","""PT"""
2025-08-20 23:00:00,"""8811361""","""MV260""","""RG640777995PT""","""""","""""","""""","""""","""""","""I""","""21/08/2025 17:35:09""",38.557668,-7.90739,"""""","""PT"""
2025-08-20 23:00:00,"""8811361""","""MV260""","""RF940510151PT""","""""","""""","""""","""""","""""","""I""","""21/08/2025 17:33:19""",38.557668,-7.90739,"""""","""PT"""
2025-08-20 23:00:00,"""8811361""","""MV260""","""RG640777911PT""","""""","""""","""""","""""","""""","""I""","""21/08/2025 17:32:02""",38.557668,-7.90739,"""""","""PT"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-08-20 23:00:00,"""8819009""","""LOCAL CTT EXPRE""","""EC213265347PT""","""""","""COOPERATIVA AGRICOLA ST. ISIDR…","""PEREIRA CALDAS 1, R/C""","""2985-158""","""SANTO ISIDRO DE PEGÕES""","""I""","""21/08/2025 08:13:11""",38.684488,-8.658045,"""CAPSULAS DO NORTE INDUSTRIAS M…","""PT"""
2025-08-20 23:00:00,"""8819009""","""LOCAL CTT EXPRE""","""EC213265333PT""","""""","""COOPERATIVA AGRICOLA ST. ISIDR…","""PEREIRA CALDAS 1, R/C""","""2985-158""","""SANTO ISIDRO DE PEGÕES""","""I""","""21/08/2025 08:13:11""",38.684488,-8.658045,"""CAPSULAS DO NORTE INDUSTRIAS M…","""PT"""
2025-08-20 23:00:00,"""8819009""","""LOCAL CTT EXPRE""","""EC213265404PT""","""""","""COOPERATIVA AGRICOLA ST. ISIDR…","""PEREIRA CALDAS 1, R/C""","""2985-158""","""SANTO ISIDRO DE PEGÕES""","""I""","""21/08/2025 08:13:11""",38.684488,-8.658045,"""CAPSULAS DO NORTE INDUSTRIAS M…","""PT"""


In [9]:
import os

# caminho para Downloads
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads", "dados.parquet")

# grava o df final em Parquet
df.write_parquet(downloads_path, compression="zstd")

print(f"Parquet gravado em: {downloads_path}")


Parquet gravado em: C:\Users\1420844\Downloads\dados.parquet
